# Obtención de datos

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/neural_machine_translation

/content/drive/MyDrive/neural_machine_translation


In [ ]:
# Clonamos el repositorio para herramientas de preprocesamiento
!git clone https://github.com/ymoslem/MT-Preparation.git

Cloning into 'MT-Preparation'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 268 (delta 133), reused 189 (delta 97), pack-reused 0
Receiving objects: 100% (268/268), 69.06 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [ ]:
# Install the requirements
!pip3 install -r MT-Preparation/requirements.txt

In [ ]:
#Descargamos el dataset
!git clone https://github.com/AmericasNLP/americasnlp2021.git

Cloning into 'americasnlp2021'...
remote: Enumerating objects: 469, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 469 (delta 89), reused 99 (delta 87), pack-reused 333
Receiving objects: 100% (469/469), 37.37 MiB | 9.79 MiB/s, done.
Resolving deltas: 100% (218/218), done.
Updating files: 100% (146/146), done.


# Filtrado de datos

In [ ]:
# Filtramos el dataset
# Arguments: source file, target file, source language, target language
!python3 MT-Preparation/filtering/filter.py americasnlp2021/data/nahuatl-spanish/dev.nah americasnlp2021/data/nahuatl-spanish/dev.es nah es
!python3 MT-Preparation/filtering/filter.py americasnlp2021/data/nahuatl-spanish/train.nah americasnlp2021/data/nahuatl-spanish/train.es nah es

Dataframe shape (rows, columns): (672, 2)
--- Rows with Empty Cells Deleted	--> Rows: 671
--- Duplicates Deleted			--> Rows: 671
--- Source-Copied Rows Deleted		--> Rows: 671
--- Too Long Source/Target Deleted	--> Rows: 613
--- HTML Removed			--> Rows: 613
--- Rows will remain in true-cased	--> Rows: 613
--- Rows with Empty Cells Deleted	--> Rows: 613
--- Rows Shuffled			--> Rows: 613
--- Source Saved: americasnlp2021/data/nahuatl-spanish/dev.nah-filtered.nah
--- Target Saved: americasnlp2021/data/nahuatl-spanish/dev.es-filtered.es
Dataframe shape (rows, columns): (16145, 2)
--- Rows with Empty Cells Deleted	--> Rows: 16062
--- Duplicates Deleted			--> Rows: 15448
--- Source-Copied Rows Deleted		--> Rows: 15373
--- Too Long Source/Target Deleted	--> Rows: 14170
--- HTML Removed			--> Rows: 14170
--- Rows will remain in true-cased	--> Rows: 14170
--- Rows with Empty Cells Deleted	--> Rows: 14170
--- Rows Shuffled			--> Rows: 14170
--- Source Saved: americasnlp2021/data/nahuatl-spanish/t

In [ ]:
!wc -l americasnlp2021/data/nahuatl-spanish/train.es-filtered.es

14170 americasnlp2021/data/nahuatl-spanish/train.es-filtered.es


# Tokenización

In [ ]:
# Juntamos los datasets de dev y train
!cat americasnlp2021/data/nahuatl-spanish/dev.nah-filtered.nah americasnlp2021/data/nahuatl-spanish/train.nah-filtered.nah > americasnlp2021/data/nahuatl-spanish/nah-es.nah
!cat americasnlp2021/data/nahuatl-spanish/dev.es-filtered.es americasnlp2021/data/nahuatl-spanish/train.es-filtered.es > americasnlp2021/data/nahuatl-spanish/nah-es.es

In [ ]:
# Entrenamos el modelo de subword tokenization
!python3 MT-Preparation/subwording/1-train_unigram.py americasnlp2021/data/nahuatl-spanish/nah-es.nah americasnlp2021/data/nahuatl-spanish/nah-es.es

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=americasnlp2021/data/nahuatl-spanish/nah-es.nah --model_prefix=source --vocab_size=50000 --hard_vocab_limit=false --split_digits=true
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: americasnlp2021/data/nahuatl-spanish/nah-es.nah
  input_format: 
  model_prefix: source
  model_type: UNIGRAM
  vocab_size: 50000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit

In [ ]:
# Subword the dataset
!python3 MT-Preparation/subwording/2-subword.py source.model target.model americasnlp2021/data/nahuatl-spanish/nah-es.es americasnlp2021/data/nahuatl-spanish/nah-es.nah

Source Model: source.model
Target Model: target.model
Source Dataset: americasnlp2021/data/nahuatl-spanish/nah-es.es
Target Dataset: americasnlp2021/data/nahuatl-spanish/nah-es.nah
Done subwording the source file! Output: americasnlp2021/data/nahuatl-spanish/nah-es.es.subword
Done subwording the target file! Output: americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword


In [ ]:
# Split the dataset into training set, development set, and test set
# Development and test sets should be between 1000 and 5000 segments (here we chose 2000)
!python3 MT-Preparation/train_dev_split/train_dev_test_split.py 2000 2000 americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword americasnlp2021/data/nahuatl-spanish/nah-es.es.subword

Dataframe shape: (14783, 2)
--- Empty Cells Deleted --> Rows: 14783
--- Wrote Files
Done!
Output files
americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.train
americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.train
americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.dev
americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.dev
americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.test
americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.test


# Configuración del modelo

In [ ]:
!pip3 install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [5]:
!pip install OpenNMT-py -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 95.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Us

In [ ]:
# Creación del archivo de configuración
# train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint
SRC_DATA_NAME = "americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword"
TARGET_DATA_NAME = "americasnlp2021/data/nahuatl-spanish/nah-es.es.subword"

config = f'''# config.yaml

## Where the samples will be written
save_data: run

# Rutas de archivos de entrenamiento
#(previamente aplicado subword tokenization)
data:
    corpus_1:
        path_src: {SRC_DATA_NAME}.train
        path_tgt: {TARGET_DATA_NAME}.train
        transforms: [filtertoolong]
    valid:
        path_src: {SRC_DATA_NAME}.dev
        path_tgt: {TARGET_DATA_NAME}.dev
        transforms: [filtertoolong]

# Vocabularios (serán generados por `onmt_build_vocab`)
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Tamaño del vocabulario
#(debe concordar con el parametro usado en el algoritmo de subword tokenization)
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filtrado sentencias de longitud mayor a n
# actuara si [filtertoolong] está presente
src_seq_length: 150
src_seq_length: 150

# Tokenizadores
src_subword_model: source.model
tgt_subword_model: target.model

# Archivos donde se guardaran los logs y los checkpoints de modelos
log_file: train.log
save_model: models/model.enfr

# Condición de paro si no se obtienen mejoras significativas
# despues de n validaciones
early_stopping: 4

# Guarda un checkpoint del modelo cada n steps
save_checkpoint_steps: 1000

# Mantiene los n ultimos checkpoints
keep_checkpoint: 3

# Reproductibilidad
seed: 3435

# Entrena el modelo maximo n steps
# Default: 100,000
train_steps: 3000

# Corre el set de validaciones (*.dev) despues de n steps
# Defatul: 10,000
valid_steps: 1000

warmup_steps: 1000
report_every: 100

# Numero de GPUs y sus ids
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0
batch_type: "tokens"
batch_size: 4096
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Configuración del optimizador
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Configuración del Modelo
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("/content/drive/MyDrive/neural_machine_translation/config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!cat /content/drive/MyDrive/neural_machine_translation/config.yaml

# config.yaml

## Where the samples will be written
save_data: run

# Rutas de archivos de entrenamiento
#(previamente aplicado subword tokenization)
data:
    corpus_1:
        path_src: americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.train
        path_tgt: americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.dev
        path_tgt: americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.dev
        transforms: [filtertoolong]

# Vocabularios (serán generados por `onmt_build_vocab`)
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Tamaño del vocabulario
#(debe concordar con el parametro usado en el algoritmo de subword tokenization)
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filtrado sentencias de longitud mayor a n
# actuara si [filtertoolong] está presente
src_seq_length: 150
src_seq_length: 150

# Tokenizadores
src_subword_model: sourc

In [ ]:
%%time
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-06-02 10:56:06,051 INFO] Counter vocab from -1 samples.
[2024-06-02 10:56:06,051 INFO] n_sample=-1: Build vocab on full datasets.
[2024-06-02 10:56:07,615 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=305)

[2024-06-02 10:56:07,618 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=328)

[2024-06-02 10:56:07,642 INFO] Counters src: 5221
[2024-06-02 10:56:07,642 INFO] Counters tgt: 5418
CPU times: user 59 ms, sys: 16.1 ms, total: 75.1 ms
Wall time: 9.04 s


In [6]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14999.0625 out of: 15102.0625


# Entrenamiento

In [ ]:
!ls

americasnlp2021  MT-Preparation  source.model  target.model
config.yaml	 run		 source.vocab  target.vocab


In [7]:
%%time
!onmt_train -config config.yaml

[2024-06-03 15:50:46,062 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-06-03 15:50:46,471 INFO] Parsed 2 corpora from -data.
[2024-06-03 15:50:46,472 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-06-03 15:50:47,306 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '▁y', '▁', 'n', '▁.', 'i', '▁,']
[2024-06-03 15:50:47,307 INFO] The decoder start token is: <s>
[2024-06-03 15:50:47,307 INFO] Building model...
[2024-06-03 15:50:48,895 INFO] Switching model to float32 for amp/apex_amp
[2024-06-03 15:50:48,895 INFO] Non quantized layer compute is fp16
[2024-06-03 15:50:49,055 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(5232, 512, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Modu

# Traducción

In [8]:
!ls models

model.enfr_step_1000.pt  model.enfr_step_2000.pt  model.enfr_step_3000.pt


In [9]:
%%time
!onmt_translate -model models/model.enfr_step_3000.pt -src americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.test -output es.translated -gpu 0 -min_length 1

[2024-06-03 15:51:51,390 INFO] Loading checkpoint from models/model.enfr_step_3000.pt
[2024-06-03 15:52:02,409 INFO] Loading data into the model
[2024-06-03 15:53:00,633 INFO] PRED SCORE: -0.3759, PRED PPL: 1.46 NB SENTENCES: 2000
Time w/o python interpreter load/terminate:  69.27337193489075
CPU times: user 452 ms, sys: 83.7 ms, total: 535 ms
Wall time: 1min 13s


In [12]:
!python MT-Preparation/subwording/3-desubword.py source.model americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.test

Done desubwording! Output: americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.test.desubword


In [13]:
!python MT-Preparation/subwording/3-desubword.py source.model es.translated

Done desubwording! Output: es.translated.desubword


In [15]:
!tail americasnlp2021/data/nahuatl-spanish/nah-es.nah.subword.test.desubword

nehhua cualli niyetoya. ¡huan yehhua yahqui nochi!
Moztla occepa niquitaz .
Isato nopa ueuentsi :
Quenotimoyohuilti , nonantzin . ¿ Quentimohuica ?
Se kinamaka tamanal ya ne Kuesalan .
yokiyakoke in lečonti .
Auh çan chiquacentetl metztli yn onmonec acalli yn nabío ynic achto nezca , necoccampa yc tepanahuilloya ; yn nican onehuaya Mexico tlaca yc onpanahuilloya ynic ompa huia in Tetzcoco , auh yn ompa Tetzcoco ompa | | 53 yc hualtepanahuiloya yn ompa huallehuaya Tetzcoco tlaca yc huel hualpanahuiloya ynic nican hualhuia Mexico , ynic yehuatl yca amo huecauh panolloya yn acalli .
Ocachi tomin quipia ca tlen itechmonequi .
Ompa▁Momochco yexpa xihuitl ohuilaloya nehuentiloz itlauhtzinco in Toteotatzin itocatzin▁Chalma . Inintzin mahuiztic teotl oconmotlazocamachililiaya ipampa tlaca mococoa , cihuatl nozo pipiltzitzintin . Zan ica oquimotenehuiliaya ocehuihualoya . Ipampa on huel miac tlacatl oquimaxilique ica▁devoción . Ihuan ica on yexpa tlaca xihuitl oanaloya . Cequi tlacatl omohuenti

In [18]:
!tail es.translated.desubword

¡Espero que tengas una mañana maravillosa! Fue lindo chatear contigo en línea
Llega a la tarde .
El viejito dijo :
¿ Cómo está su madre , ?
Se vende en Cuetzalan .
Ya no los mata a sus marranos .
A éste lo sorprendió una tempestad muy fuerte que soplaba del oriente y duró mucho tiempo , de modo que no pudieron aportar en ningún sitio ; pero con su habilidades .
Tendrás dinero .
Y el señor de Chalma nos devolvió a nuestra mamá . Así es como nos devolvió a nuestra madre . Todavía vivió mucho . Y todos los años íbamos en peregrinación hasta los pies del Dios . Primero fuimos así : no llevamos nada . Y al siguiente año llevamos dos candeleras y ceras y flores como ofrenda por la salud de nuestra madre .
Si no hay tomate se puede comer , se muele .


# Evaluación

In [19]:
!git clone https://github.com/ymoslem/MT-Evaluation.git

Cloning into 'MT-Evaluation'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 101 (delta 46), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (101/101), 20.17 KiB | 1.44 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [7]:
!pip install -r MT-Evaluation/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 61.9 MB/s eta 0:00:00


In [21]:
# Aplicando desubword para tener la traducción de referencia
!python MT-Preparation/subwording/3-desubword.py target.model americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.test

Done desubwording! Output: americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.test.desubword


BLEU

In [27]:
!python MT-Evaluation/BLEU/compute-bleu.py americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.test.desubword es.translated.desubword

Reference 1st sentence: Donde hay árboles y ríos , durante todo el verano la juventud se divierte bañándose todos los días .
MTed 1st sentence: Aquí están los niños , los que van en los caballos podridos .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU:  6.926355150258824


BLEU: 6.926355150258824

ChrF

In [10]:
!python americasnlp2021/evaluate.py --sys americasnlp2021/data/nahuatl-spanish/nah-es.es.subword.test.desubword --ref es.translated.desubword

[]
2000
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
#### Score Report ####
chrF2 = 32.06
BLEU = 6.99


chrF2 = 32.06

# Comparación con baseline

Mis resultados en BLEU fueron entre 6.92, en ChrF fue de 32.06.

Los resultados de baseline en BLUE fueron 0.33 y en ChrF de 0.182.

Por lo tanto el modelo que hice tuvo mejores resultados que el baseline en ambas métricas, pero más en ChrF.

# Extra

Investigar porque se propuso la medida ChrF en el Shared Task.

ChrF es una métrica a nivel de caracteres. Se propuso porque para idiomas con mucha morfología, las métricas a nivel palabras son muy problemáticas.

¿Como se diferencia de BLEU?

ChrF se basa en comparar similitud entre n-gramas de caracteres, mientras que BLEU lo compara entre n-gramas de palabras.

¿Por qué es relevante utilizar otras medidas de evaluación además de BLEU?

Porque distintos idiomas tienen propiedades distintas. Para idiomas con mucha morfología, dos palabras pueden tener casi los mismos morfemas aunque sean palabras diferentes; eso no se capturaría en una métrica a nivel palabra como BLEU, pero sí en una a nivel caracter.